In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hipp
from getpass import getpass
import glob
import os

## Login

In [ ]:
apiKey = hipp.dataquery.EE_login(input(), getpass())

## Inputs

In [ ]:
bounds = (-122, 49, -121.5, 48.5)
ee_project_name = 'LK000'
year = 1950
month = 9
day = 2
output_directory    = './'
ee_query_max_results   = 10 #do more than 1 because you may get a calibration file
ee_query_label = 'test_download' #try to use the same one for consistent behavior from the EE API

## Query EE

and filter results by the project name you are interested in (or look at the entire ee_results_df first)

In [ ]:
ULLON, ULLAT, LRLON, LRLAT = bounds

ee_results_df = hipp.dataquery.EE_pre_select_images(
    apiKey,
    xmin = LRLON,
    ymin = LRLAT,
    xmax = ULLON,
    ymax = ULLAT,
    startDate = f"{year}-{month}-{day}",
    endDate = f"{year}-{month}-{day}",
    maxResults   = ee_query_max_results
)

ee_results_df = ee_results_df[ee_results_df['project'] == ee_project_name]

In [ ]:
ee_results_df

## Select the first image for download and download it
Ideally you want to select an image that is representative of the set, not necessarily the first one.
(e.g. choose and image without over or under exposed areas)

In [ ]:
ee_results_df = ee_results_df.head(1)

In [ ]:
images_directory, calibration_reports_directory = hipp.dataquery.EE_download_images_to_disk(
    apiKey,
    ee_results_df['entityId'].tolist(),
    ee_query_label,
    output_directory
)
single_image_file = glob.glob(os.path.join(images_directory,'*.tif'))[0]

## Create fiducial templates from images

Note that the optional parameter buffer_distance should be tuned to the dataset you are using.

In [ ]:
fiducial_template_directory = os.path.join(output_directory, 'fiducials')

hipp.core.create_midside_fiducial_proxies_template(
    single_image_file,
    fiducial_template_directory,
    buffer_distance = 250 #default value

)

## Detect principal points and crop images

In [ ]:
hipp.batch.preprocess_with_fiducial_proxies(images_directory,
                                            fiducial_template_directory)